In [473]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(1, os.path.dirname(os.getcwd()))

# from copy import deepcopy
# import torch.optim as optim
# import torch
from scipy import fft
import numpy as np
from utils.network_graph import DirectedGraph, graph_dict_3x3, graph_dict_3x3_2, graph_dict_4_nodes


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [474]:
graph = DirectedGraph(graph_dict_3x3_2)
# graph = DirectedGraph(graph_dict_4_nodes)


In the next cell you can see the process of restoring od-flows from o-flows and getting link flows using both O and OD flows and assignment matrices. Fortunately, the results are equivalent.

In [475]:
o_flows, o_nodes = graph.generate_o_flows()
# print(fft.dct(o_flows, type=1, norm="forward"))
A_od = graph.generate_od_assignment_matrix()
P_o = graph.generate_o_assignment_matrix()

od_flows, od_pairs = graph.get_od_from_o_flows(o_flows, P_o)

print("P_o.shape:", P_o.shape, end="; ")
print("A_od.shape:", A_od.shape, end="\n\n")

print("o flows: \n", o_flows)
print("od flows: \n", np.round(od_flows))

print("\nlink flows from o: \n", np.round(P_o @ o_flows))
print("link flows from od: \n", np.round(A_od @ od_flows))

real_link_flows = P_o @ o_flows

P_o.shape: (24, 9); A_od.shape: (24, 72)

o flows: 
 [457 663 902 303 223 201 199 873 229]
od flows: 
 [ 52.  32.  10.  95.  57.  74.  97.  39. 101. 149.  74.   8. 142.  87.
  16.  86.  34. 118.  34. 120. 181. 177.  42. 197.  39.  43.   5.  63.
  27.  63.  26.  36.  16.  26.   4.  33.  24.  18.  48.  53.   8.  40.
  10.  32.  24.   2.  44.  41.   5.  34.  40.  29.  33.  38.   5.  14.
  61.  64. 132.  12.  39. 162. 129. 274.  47.  40.   8.  20.   3.  41.
  17.  53.]

link flows from o: 
 [304. 369. 320. 390. 629. 464. 674. 208. 379. 387. 328. 348. 518. 480.
 225. 356. 422. 198. 196. 473. 374. 586. 283. 215.]
link flows from od: 
 [304. 369. 320. 390. 629. 464. 674. 208. 379. 387. 328. 348. 518. 480.
 225. 356. 422. 198. 196. 473. 374. 586. 283. 215.]


In [476]:
# example solution of a task posed in
# https://scaron.info/blog/quadratic-programming-in-python.html

from utils.opt_functions import quadprog_solve_qp

M = np.array([[1., 2., 0.], [-8., 3., 2.], [0., 1., 1.]])
P = np.dot(M.T, M)
q = -np.dot(M.T, np.array([3., 2., 3.]))
G = np.array([[1., 2., 1.], [2., 0., 1.], [-1., 2., -1.]])
h = np.array([3., 2., -2.])

quadprog_solve_qp(P, q, G, h)

# array([ 0.12997347, -0.06498674,  1.74005305])

array([ 0.12997347, -0.06498674,  1.74005305])

In [477]:
# optimize o-flows with a known assignment matrix
M = P_o
# M = P_torch.detach().numpy()
P = np.dot(M.T, M)
q = -np.dot(M.T, P_o @ o_flows) # P_o @ o_flows is a vector of link flows
G = -np.ones((len(o_flows), len(o_flows))) # C1: o-flows should be positive
h = np.zeros(len(o_flows)) 

print("o-flows opt solution:", quadprog_solve_qp(P, q, G, h))
print("o-flows ground truth:", o_flows)

o-flows opt solution: [457. 663. 902. 303. 223. 201. 199. 873. 229.]
o-flows ground truth: [457 663 902 303 223 201 199 873 229]


In [561]:
links = graph.get_links()

n_t = 15 # number of intervals
num_o_nodes = len(o_nodes)
num_links = len(links)

o_flows_mat = np.zeros((num_o_nodes, n_t))
# link_flows = P_o @ o_flows
link_flows_mat = np.zeros((num_links, n_t))
for i in range(n_t):
    o_flows = graph.generate_o_flows()[0]
    link_flows = P_o @ o_flows

    o_flows_mat[:, i] = o_flows
    link_flows_mat[:, i] = link_flows


r, c = P_o.shape

M = np.zeros((r * n_t, r*c))
for j in range(n_t):
    for i in range(r):
        M[i + j * r, i*c:i*c+c] = o_flows_mat[:, j]


P = np.dot(M.T, M)
b = link_flows_mat.T.reshape(np.prod(link_flows_mat.shape))

q = -M.T @ b
# q = np.zeros(r * c)
# for i in range(r):
#     q[i * c: i * c + c] = link_flows[i] * o_flows


C2_mat = np.vstack([np.eye(r*c), -np.eye(r * c)]) 
C2_vect = np.hstack([np.ones(r*c), np.zeros(r * c)])

C3_mat = np.zeros((c, r * c))
for j in range(c):
      for i in range(r):
            if (int(links[i][0])) == j + 1:
                  C3_mat[j, i * c + j] = 1
C3_vect = np.ones(c)


G = C2_mat
h = C2_vect
A = C3_mat
b = C3_vect

# P += np.eye(P.shape[0]) * np.random.rand(216) * 1e-8

ans = (quadprog_solve_qp(P, q, G, h, A, b))
np.round(ans * 1000)

array([503.,  -0.,   0., 108.,  13.,  20., 107.,  15.,   0., 497., 116.,
        58.,  -0.,  22.,  32.,   0.,   1.,  -0.,   0., 253.,  85.,  13.,
        58.,  58.,   8.,  25., 103., 126., 349.,   0.,  29.,  42.,   5.,
       100.,  69.,  10., 283., 398., 234.,  36.,   0.,  49.,  17.,   0.,
         1.,   0.,  -0., 432.,  14.,  18., 157.,  -0.,  15.,  94.,  77.,
       146., 568.,  18.,  33.,  -0.,   0.,  19.,   0.,   0.,  15.,  11.,
       225.,  49.,  33., 127.,  61., 103., 279.,  25.,  13., 463.,   0.,
         0., 281.,  30.,   1., 222.,  92.,  98., 312.,  84.,  25.,   0.,
        15.,  21.,  21.,   0.,  18.,  99., 188., 133., 189., 138., 196.,
        25., 117., 101.,  -0., 240., 153.,  19.,  53., 147., 132., 175.,
        51., 122., 235.,   0., 215., 182.,  23., 216., 133., 159., 112.,
       337., 149.,  12.,   0.,  34.,  20.,  23.,   0.,  20.,  27., 203.,
       100., 115., 120.,  21.,  15., 201.,   0.,  -0., 460.,  17.,  13.,
       210.,  42.,  91., 218.,  48., 146., 336.,  2

In [524]:
# for i in range(10):
#     print(graph.generate_o_flows()[0])
np.round(ans.reshape(P_o.shape) @ o_flows_mat[:, 0])

array([229., 194., 147., 182., 182., 147., 100., 206., 288., 224., 377.,
       277., 364., 310., 265., 311., 261., 204., 179., 463., 269., 455.,
       356., 302.])

In [583]:
np.round(link_flows_mat[:, 0])

array([520., 388., 182., 226., 344., 245., 217., 398., 785., 530., 484.,
       306., 444., 479., 274., 401., 296., 349., 446., 425., 227., 308.,
       506., 514.])

In [682]:
# try the overall procedure
graph2 = DirectedGraph(graph_dict_3x3_2)
P_init = graph2.generate_o_assignment_matrix()
np.round(P_init @ o_flows_mat[:, 0])

opt_P = P_init
opt_x = np.zeros((num_o_nodes, n_t))


In [689]:
epochs = 50
# opt_P = P_init
for i in range(epochs):
    # optimize x

    M_x = opt_P
    P_x = np.dot(M_x.T, M_x)
    G_x = np.vstack([-np.eye(num_o_nodes) for i in range(num_o_nodes)])
    
    h_x = np.zeros(num_o_nodes ** 2)
    for j in range(n_t):
        q_x = -np.dot(M_x.T, link_flows_mat[:, j]) # P @ o_flows_mat is a matirx of link flows
        opt_x[:, j] = quadprog_solve_qp(P_x, q_x, G_x, h_x)


    #optimize P
    r, c = P_o.shape

    M = np.zeros((r * n_t, r*c))
    for j in range(n_t):
        for i in range(r):
            M[i + j * r, i*c:i*c+c] = opt_x[:, j]

    # P = M.T @ M
    # q = -M.T @ b
    P = np.dot(M.T, M)
    b = link_flows_mat.T.reshape(np.prod(link_flows_mat.shape))

    q = -M.T @ b

    C2_mat = np.vstack([np.eye(r*c), -np.eye(r * c)]) 
    C2_vect = np.hstack([np.ones(r*c), np.zeros(r * c)])

    C3_mat = np.zeros((c, r * c))
    for j in range(c):
        for i in range(r):
                if (int(links[i][0])) == j + 1:
                    C3_mat[j, i * c + j] = 1
    C3_vect = np.ones(c)


    G = C2_mat
    h = C2_vect
    A = C3_mat
    b = C3_vect


    opt_P = (quadprog_solve_qp(P, q, G, h, A, b)).reshape(P_o.shape)

In [690]:
np.round(opt_x[:, 0]), np.round(o_flows_mat[:, 0])

(array([683., 100., 237., 989., 405., 278., 513.,  32., 748.]),
 array([700.,  87., 216., 945., 273., 335., 506., 100., 846.]))

In [1]:
# np.round(opt_P, 3) * 1000, np.round(P_o, 3) * 1000